[![Fixel Algorithms](https://fixelalgorithms.co/images/CCExt.png)](https://fixelalgorithms.gitlab.io)

# Scientific Programming

## Dynamic Programming - Piece Wise Linear Regression

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 1.0.000 | 01/05/2025 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/AIProgram/2024_02/0092DeepLearningResNet.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Machine Learning
from sklearn.linear_model import LinearRegression

# Deep Learning

# Miscellaneous
from enum import auto, Enum, unique
import math
import os
from platform import python_version
import random

# Typing
from typing import Callable, Dict, Generator, List, Optional, Self, Set, Tuple, Union

# Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Jupyter
from IPython import get_ipython


## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

Code Notations:

```python
someVar    = 2; #<! Notation for a variable
vVector    = np.random.rand(4) #<! Notation for 1D array
mMatrix    = np.random.rand(4, 3) #<! Notation for 2D array
tTensor    = np.random.rand(4, 3, 2, 3) #<! Notation for nD array (Tensor)
tuTuple    = (1, 2, 3) #<! Notation for a tuple
lList      = [1, 2, 3] #<! Notation for a list
dDict      = {1: 3, 2: 2, 3: 1} #<! Notation for a dictionary
oObj       = MyClass() #<! Notation for an object
dfData     = pd.DataFrame() #<! Notation for a data frame
dsData     = pd.Series() #<! Notation for a series
hObj       = plt.Axes() #<! Notation for an object / handler / function handler
```

### Code Exercise

 - Single line fill

```python
vallToFill = ???
```

 - Multi Line to Fill (At least one)

```python
# You need to start writing
?????
```

 - Section to Fill

```python
#===========================Fill This===========================#
# 1. Explanation about what to do.
# !! Remarks to follow / take under consideration.
mX = ???

?????
#===============================================================#
```

In [ ]:
# Configuration
# %matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# Color Palettes
lMatPltLibclr   = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'] #<! Matplotlib default color palette
lFlexokiClr     = ['#D14D41', '#DA702C', '#D0A215', '#879A39', '#3AA99F', '#4385BE', '#8B7EC8', '#CE5D97'] #<! Flexoki (Obsidian) Main
lFlexokiSatClr  = ['#AF3029', '#BC5215', '#BC5215', '#66800B', '#24837B', '#205EA6', '#5E409D', '#A02F6F'] #<! Flexoki (Obsidian) Saturated
lFlexokiGrayClr = ['#100F0F', '#1C1B1B', '#282726', '#343331', '#403E3C', '#55524E', '#878580', '#CECDC3'] #<! Flexoki (Obsidian) Grayscale
# sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())


In [ ]:
# Constants



In [ ]:
# Courses Packages



In [ ]:
# General Auxiliary Functions

def SetDiagonal(mA: np.ndarray, k: int, valA: np.number) -> np.ndarray:
    """
    Set the k-th diagonal of matrix `mA` to the given `value`.
    Input:
        - mA: The matrix to modify.
        - k: The diagonal index. Positive for upper diagonals, negative for lower diagonals.
        - value: The value to set on the diagonal.
    Output:
        - mA: np.ndarray - The modified matrix with the specified diagonal set to `value`.
    Example:
        ```python
        mA = np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]])
        mA = SetDiagonal(mA, 1, 0)  #<! Sets the first upper diagonal to 0
        mA = SetDiagonal(mA, -1, 0) #<! Sets the first lower diagonal to 0
        print(mA)
        ```
    Remarks:
        - The function modifies the input matrix `mA` in place.
        - The diagonal index `k` can be positive (for upper diagonals) or negative (for lower diagonals).
        - For NumPy number class see https://numpy.org/doc/stable/reference/arrays.scalars.html.
    """
    numRows, numCols = mA.shape
    if k >= 0:
        diagLen = min(numRows, numCols - k)
        vRowIdx = np.arange(diagLen)
        vColIdx = vRowIdx + k
    else:
        diagLen = min(numRows + k, numCols)
        vRowIdx = np.arange(diagLen) - k
        vColIdx = np.arange(diagLen)

    mA[vRowIdx, vColIdx] = valA

    return mA


## Piece Wise Regression

Assume the following data samples: $\left\{ {x}_{i}, {y}_{i} \right\}_{i = 1}^{N}$:

![](https://i.imgur.com/3VaO2je.png)
<!-- ![](https://i.postimg.cc/G3KV8wWL/TMP.png) -->

Without prior knowledge on the _knots_, the breakpoints between segments, using _Linear Regression_ will fail.

* <font color='brown'>(**#**)</font> There are models to solve the problem using a Convex Optimization.

In [ ]:
# Parameters

fileUrl = r'https://raw.githubusercontent.com/FixelAlgorithmsTeam/FixelCourses/refs/heads/master/DataSets/PieceWiseLinearData.csv'

# Data
decFactor  = 1 #<! Decimation factor

# Model
segRadius = 0
λ         = 1.0


### Linear Regression

This section shows the result of _Naive_ use of linear regression for the data.

In [ ]:
# Load Data

dfData = pd.read_csv(fileUrl)


In [ ]:
# Plot Linear Model

with plt.style.context('dark_background'):
    hF, hA = plt.subplots(figsize = (10, 4))
    sns.set_theme(style = 'ticks', context = 'talk')
    sns.regplot(data = dfData, x = 'x', y = 'y', ax = hA, order = 1, ci = None, label = 'Samples', scatter_kws = dict(color = lFlexokiClr[0]), line_kws = dict(color = lFlexokiClr[1], label = 'Linear Fit'))
    hA.legend()
    hA.set_title('Linear Model');

### Dynamic Programming

In order to solve the problem as a Dynamic Programming problem one must define a cost function.  
A simple measure to the quality of the regression over a section could the be the ${R}^{2}$ Score ([Coefficient of Determination](https://en.wikipedia.org/wiki/Coefficient_of_determination)).

The model should be the solution given an optimal solution to a sub problem.

## Problem Statement

Given a set of points $\left\{ {x}_{i}, {y}_{i} \right\}_{i = 1}^{N}$ with up to $L$ segments find the optimal segmentation of teh data such that the sum of ${R}^{2}$ over all segments is minimized.

* <font color='brown'>(**#**)</font> It is assumed that ${x}_{i + 1} > {x}_{i}$.

### Dynamic Programming Solution

 * Each sample can be in a Single Segment.  
   This allow handling non continuous cases.
 * Define ${S}_{L} \left( i, j \right)$ to be the cost of having a segment $i \to j$ with the previous segment ending at $i - 1$.
 * Initialization $S \left( 1, j \right) = \boldsymbol{C}_{1, j}$.
 * The objective $min_{k \in \left\{ 1, 2, 3, \ldots, N - 1 \right\}} S \left( k, N \right)$.


The cost $C \left( i, j \right)$ will be defined as the ${R}^{2}$ score over the segment $i \to j$.

The recursion is given by:

$$
S \left( i, j \right) = \begin{cases} \min_{k \in \left\{ 1, 2, \ldots, i - 2 \right\}} S \left( k, i - 1 \right) + \boldsymbol{C}_{i, j} + \lambda R \left( i, j \right) & \text{ if } i < j \\ \infty & \text{ if } i \geq j \end{cases}
$$

Where $\lambda R \left( i, j \right)$ is a regularization term which promotes longer segments by excessive cost for short segments.

<!-- White Template -->
<!-- ![](https://i.imgur.com/Qc1dWVp.png) -->
<!-- ![](https://i.postimg.cc/j51qFmjV/Diagrams-Segmented-Regression-001.png) -->

<!-- Black Template -->
![](https://i.imgur.com/3WxtENM.png)
<!-- ![](https://i.postimg.cc/hjsZ4Rdw/Diagrams-Segmented-Regression-001.png) -->

The idea is to analyze the signal from the end:

 * Check all segments from sample $i \in \left\{ 1, 2, 3, \ldots, N - 1 \right\}$ to sample $j = N$.
 * For each segment from above, reapply the algorithm for segments $i \in \left\{ 1, 2, 3, \ldots, N - 3 \right\}$ to $j = N - 2$.
 * Repeat.

This recursive structure can find teh optimal segmentation.


In [ ]:
def CalcCostMatrix( mX: np.ndarray ) -> np.ndarray:
    """
    Calculate the cost matrix for a given set of data points.  
    The cost at position (i, j) is defined as R2 score of the linear regression model fitted to the data points i through j.
    Input:
        - mX: The input data points (numSamples, 2).
    Output:
        - mC: The cost matrix (numSamples, numSamples).
    Example:
        ```python
        mX = np.array([[1, 2], [3, 4], [5, 6], [7, 8], [9, 10]])
        mC = CalcCostMatrix(mX)
        print(mC)
        ```
    Remarks:
        - The cost matrix is symmetric.
        - The diagonal of the cost matrix is set to 1, as the R2 score of a single point is always 1.
        - Avoid `NaN` values so `maximum()` and `minimum()` functions can be used.
    """
    
    numSamples = mX.shape[0]
    mC         = np.empty((numSamples, numSamples)) #<! Initialize the cost matrix with NaN values
    np.fill_diagonal(mC, 1) #<! Set the diagonal to 1 (R2 score of a single point is always 1)

    oLinReg = LinearRegression(fit_intercept = True) #<! Initialize the linear regression model
    
    # Calculate the cost matrix
    for ii in range(numSamples):
        for jj in range(ii + 1, numSamples):
            vX = mX[ii:(jj + 1), 0]
            vY = mX[ii:(jj + 1), 1]
            # Using -R2 score to make it a cost function (Lower is better)
            mC[ii, jj] = -oLinReg.fit(vX[:, None], vY).score(vX[:, None], vY) #<! Fit the model and calculate the R2 score
            mC[jj, ii] = mC[ii, jj] #<! Symmetric matrix
    
    return mC

In [ ]:
def CalcMinCostSegmentation( mC: np.ndarray, maxSegments: int, /, *, λ: float = 0.0 ) -> Tuple[np.ndarray, np.ndarray]:
    """
    Calculate the minimum cost segmentation of a given cost matrix.
    The function uses dynamic programming to find the optimal segmentation of the cost matrix.
    Input:
        - mC: Cost matrix (2D numpy array) where mC[i, j] is the cost of segmenting the data from i to j.
        - maxSegments: Maximum number of segments to consider for segmentation.
    Output:
        - mS: Matrix of minimum costs for each segment.
        - mP: Matrix of paths for each segment. The path matrix indicates the previous segment for each segment.
    Example:
        ```python
        mC = np.array([[0, 1, 2], [1, 0, 1], [2, 1, 0]])
        maxSegments = 2
        mS, mP = CalcMinCostSegmentation(mC, maxSegments)
        print(mS)
        print(mP)
        ```
    Remarks:
        - This function is a simplified version of the dynamic programming algorithm for minimum cost segmentation.
    """

    numSamples  = mC.shape[0]
    maxSegments = min(maxSegments, numSamples)
    mS = np.full((maxSegments, numSamples), np.max(mC))   #<! Cost per segment
    mP = np.zeros((maxSegments, numSamples), dtype = int) #<! Path matrix

    mS[0, :] = mC[0, :] #<! Cost of the first segment is the cost of the first sample to all other samples
    mP[0, :] = 0        #<! Path for the first segment is always 1 (no previous segment)

    for ii in range(1, maxSegments):
        for jj in range(ii + 1, numSamples):
            minCost = 1e50
            kkMin   = 0
            for kk in range(ii):
                currCost = mS[kk, ii - 1] + mC[ii, jj] - λ * abs(jj - ii) #<! No need for `abs()` since `jj` is always greater than `ii`
                if currCost < minCost:
                    minCost = currCost
                    kkMin   = kk
            mS[ii, jj] = minCost
            mP[ii, jj] = kkMin
    
    return mS, mP

In [ ]:
# Extract the Path from `mP` Matrix

def ExtractPath( mP: np.ndarray, mS: np.ndarray ) -> List[int]:
    """
    Extract the path from the path matrix `mP` using the minimum cost segmentation matrix `mS`.
    The function starts from the last segment and traces back to the first segment using the path matrix.
    Input:
        - mP: Path matrix (2D numpy array) where mP[i, j] is the previous segment for segment i at sample j.
        - mS: Minimum cost segmentation matrix (2D numpy array) where mS[i, j] is the minimum cost for segment i at sample j.
    Output:
        - lPath: List of indices representing the path from the first segment to the last segment.
    Example:
        ```python
        mP = np.array([[0, 1, 2], [1, 0, 1], [2, 1, 0]])
        mS = np.array([[0, 1, 2], [1, 0, 1], [2, 1, 0]])
        lPath = ExtractPath(mP, mS)
        print(lPath)
        ```
    Remarks:
        - This function assumes that the last segment is always valid and starts from there.
    """

    numSegments = mP.shape[0]
    numSamples  = mP.shape[1]

    lPath = []
    startIdx = np.argmin(mS[:, -1]) #<! Find the starting index of the last segment
    endIdx   = numSamples - 1 #<! The last sample is the end of the last segment

    while startIdx >= 0 and endIdx >= 0:
        lPath.append((startIdx, endIdx))
        coldIdx = mP[startIdx, endIdx]
        endIdx = startIdx - 1
        startIdx = coldIdx
    
    return lPath[::-1] #<! Reverse the path to get it in correct order

In [ ]:
# Calculate the Cost Matrix
mX = dfData.to_numpy()
# mX[:, 1] = mX[:, 1] + 0.1 * np.random.randn(mX.shape[0]) #<! Add noise to the y values
mC = CalcCostMatrix(mX[::decFactor, :]) 

In [ ]:
# Adjust Cost Matrix

for dd in range(-segRadius, segRadius + 1):
    mC = SetDiagonal(mC, dd, 1e6) #<! Set the diagonals to 0

mC

In [ ]:
mS, mP = CalcMinCostSegmentation(mC, 200, λ = λ) #<! R2 is a score, so we want to minimize the cost, hence the negative sign
mS

In [ ]:
for dd in range(-segRadius, segRadius + 1):
    mC = SetDiagonal(mC, dd, math.nan) #<! Set the diagonals to 0

In [ ]:
hF, hA = plt.subplots(figsize = (5, 5))
sns.heatmap(mC, annot = False, cbar = False, ax = hA)
plt.show()

* <font color='brown'>(**#**)</font> For the data above, can the segments be inferred from the Cost Matrix?

In [ ]:
hF, hA = plt.subplots(figsize = (5, 5))
sns.heatmap(mS, annot = False, cbar = True, ax = hA)
plt.show()

In [ ]:
# Extract the Path
lPath = ExtractPath(mP, mS)
lPath

In [ ]:
# Assign Segment Label

lS = []
for ii in range(len(lPath)):
    lS.extend([ii] * (lPath[ii][1] - lPath[ii][0] + 1))

In [ ]:
# Plot the Segmentation

with plt.style.context('dark_background'):
    hF, hA = plt.subplots(figsize = (10, 4))
    sns.set_theme(style = 'ticks', context = 'talk')
    sns.scatterplot(data = dfData, x = 'x', y = 'y', ax = hA, c = lS)
    hA.set_title('Segmented Data')

    plt.show()

* <font color='green'>(**@**)</font> Write the Dynamic Programming method for the case the minimal segment length is given by $L$.